In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata/

Cloning into 'yemoonsaBigdata'...


# 작업형 제 1유형

## 다음은 Boston Housing 데이터 세트이다. 범죄율 컬럼인 Crim 항목의 상위에서 10번째 값(즉, 범죄율을 큰 순서로 내림차순 정렬했을 때 10번째에 위치한 값)으로 상위 10개의ㅣ 값을 변환한 수 age가 80 이상인 데이터를 추출하여 crim의 평균값을 계산하시오.

In [3]:
import pandas as pd
df = pd.read_csv('yemoonsaBigdata/datasets/Part3/201_boston.csv')
res_df = df.sort_values('CRIM', ascending = False).reset_index(drop = True)
value = res_df.iloc[9,0]
res_df.iloc[:10, 0] = value
res = res_df[res_df['AGE'] >= 80]['CRIM'].mean()
print(res)

5.759386625


## 다음은 California housing데이터 세트이다. 주어진 데이터의 첫 번째 행부터 순서대로 80%까지의 데이터를 훈련 데이터로 추출 후, 전체 방 개수 컬럼을 의미하는 total bedrooms변수의 결측치를 total_bedrooms 변수의 중앙값으로 대체한 데이터 세트를 구성한다. 결측치 대체 전의 total_bedrooms 변수 표준편차 값과 결측치 대체 후의 total_bedrooms 뱐수 표준편차 값의 차이에 대한 절댓값을 계산하시오

In [25]:
import pandas as pd
df = pd.read_csv('yemoonsaBigdata/datasets/Part3/202_housing.csv')
# 전체 80%
ind = int(len(df) * (80 / 100))
train_df = df.loc[:ind, 'total_bedrooms']
b_std = train_df.std()
train_df = train_df.fillna(train_df.median())
a_std = train_df.std()
res = abs(b_std - a_std)
print(res)

1.9749908183319462


## 2번 문항에서 활용한 데이터 세트를 다시 이용. 인구 컬럼인 population 항목의 이상값의 합계를 계산(평균에서 1.5 * 표준편차를 초과하거나 미만인 값의 범위)

In [31]:
import pandas as pd
df = pd.read_csv('yemoonsaBigdata/datasets/Part3/202_housing.csv')

res_df = df[(df['population'] > df['population'].mean() + 1.5 * df['population'].std()) | (df['population'] < df['population'].mean() - 1.5 * df['population'].std())]
res = res_df['population'].sum()
print(res)

5607295.0


# 작업형 제 2유형

다음은 E-commerce shipping 데이터 세트이다. 주어진 훈련 데이터 세트를 활용하여 고객이 주문한 물품의 정시 도착 여부를 예측하고 해당 예측 결과를 csv 파일로 제출(roc-auc로(확률))

In [101]:
import pandas as pd
x_test = pd.read_csv('yemoonsaBigdata/datasets/Part3/204_x_test.csv')
x_train = pd.read_csv('yemoonsaBigdata/datasets/Part3/204_x_train.csv')
y_train = pd.read_csv('yemoonsaBigdata/datasets/Part3/204_y_train.csv')


In [58]:
x_train[col_cat].nunique()

Gender                2
Mode_of_Shipment      3
Product_importance    3
Warehouse_block       5
dtype: int64

## 방법 1. ohehotencoder

In [102]:
# 결측치, 이상값 X
# 명목형
col_drop = ['ID']
col_num = ['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']
col_cat = ['Gender', 'Mode_of_Shipment', 'Product_importance', 'Warehouse_block']

x_train = x_train.drop(col_drop, axis = 1)
x_test = x_test.drop(col_drop, axis = 1)
y_train = y_train.drop(col_drop, axis = 1)

x = pd.concat([x_train, x_test])

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(x[col_cat])

x_train_res = ohe.transform(x_train[col_cat])
x_test_res = ohe.transform(x_test[col_cat])

x_train_ohe = pd.DataFrame(x_train_res.todense(), columns = ohe.get_feature_names_out())
x_test_ohe = pd.DataFrame(x_test_res.todense(), columns = ohe.get_feature_names_out())

x_train_fin = pd.concat([x_train_ohe, x_train[col_num]], axis = 1)
x_test_fin = pd.concat([x_test_ohe, x_test[col_num]], axis = 1)

# split
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train_fin, y_train.values.ravel(), test_size = 0.3, stratify = y_train.values.ravel())

# scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test_fin[col_num] = scaler.transform(x_test_fin[col_num])

# model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth = 10, min_samples_split = 5, min_samples_leaf  = 2, max_features = 'sqrt')
model.fit(x_tr, y_tr)

from sklearn.metrics import accuracy_score
y_pred_train = model.predict(x_tr)
print('train as : ', accuracy_score(y_pred_train, y_tr))

y_pred_val = model.predict(x_val)
print('val as : ', accuracy_score(y_pred_val, y_val))

from sklearn.metrics import roc_auc_score
y_pred_train_proba = model.predict_proba(x_tr)[:, 1]
print('train roc : ', roc_auc_score(y_tr, y_pred_train_proba))

y_pred_val_proba = model.predict_proba(x_val)[:, 1]
print('val roc : ', roc_auc_score(y_val, y_pred_val_proba))

y_pred = model.predict_proba(x_test_fin)[:, 1]
pd.DataFrame({'index' : x_test.index, 'y_pred' : y_pred}).to_csv('00300.csv', index = False)

train as :  0.872916215631089
val as :  0.6797979797979798
train roc :  0.9710300738465901
val roc :  0.7595341992884072


## 방법 2. labelencoder

In [94]:
import pandas as pd
x_test = pd.read_csv('yemoonsaBigdata/datasets/Part3/204_x_test.csv')
x_train = pd.read_csv('yemoonsaBigdata/datasets/Part3/204_x_train.csv')
y_train = pd.read_csv('yemoonsaBigdata/datasets/Part3/204_y_train.csv')

# 결측치, 이상값 X
col_drop = ['ID']
col_num = ['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']
col_cat = ['Gender', 'Mode_of_Shipment', 'Product_importance', 'Warehouse_block']

x_train = x_train.drop(col_drop, axis = 1)
x_test = x_test.drop(col_drop, axis = 1)
y_train = y_train.drop(col_drop, axis = 1)


# split
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train.values.ravel(), test_size = 0.3, stratify = y_train.values.ravel())

# scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test[col_num] = scaler.transform(x_test[col_num])

# 명목형
from sklearn.preprocessing import LabelEncoder
x = pd.concat([x_train[col_cat], x_test[col_cat]])

for col in col_cat:
    le = LabelEncoder()
    le.fit(x[col])
    x_tr[col] = le.transform(x_tr[col])
    x_val[col] = le.transform(x_val[col])
    x_test[col] = le.transform(x_test[col])

# model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_tr, y_tr)

from sklearn.metrics import accuracy_score
y_pred_train = model.predict(x_tr)
print('train as : ', accuracy_score(y_pred_train, y_tr))

y_pred_val = model.predict(x_val)
print('val as : ', accuracy_score(y_pred_val, y_val))

from sklearn.metrics import roc_auc_score
y_pred_train_proba = model.predict_proba(x_tr)[:, 1]
print('train roc : ', roc_auc_score(y_tr, y_pred_train_proba))

y_pred_val_proba = model.predict_proba(x_val)[:, 1]
print('val roc : ', roc_auc_score(y_val, y_pred_val_proba))

y_pred = model.predict_proba(x_test)[:, 1]
pd.DataFrame({'index' : x_test.index, 'y_pred' : y_pred}).to_csv('00300.csv', index = False)

train as :  1.0
val as :  0.6727272727272727
train roc :  1.0
val roc :  0.747793235203067
